In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

import nlpsig_networks
from nlpsig_networks.ffn import FeedforwardNeuralNetModel
from nlpsig_networks.focal_loss import FocalLoss
from nlpsig_networks.pytorch_utils import training_pytorch, testing_pytorch

seed = 2023

In [2]:
batch_size = 100

# MNIST
train_dataset = torchvision.datasets.MNIST(root="./data", train=True,
                                           transform=transforms.Compose([transforms.ToTensor(),
                                                                         transforms.Lambda(lambda x: torch.flatten(x))]),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False,
                                          transform=transforms.Compose([transforms.ToTensor(),
                                                                        transforms.Lambda(lambda x: torch.flatten(x))]),
                                          download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,
                                           shuffle=True,)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,
                                           shuffle=False)

/Users/rchan/opt/miniconda3/envs/nlpsig-networks/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
examples = iter(train_loader)
samples, label = examples.next()
print(samples.shape,label.shape)

torch.Size([100, 784]) torch.Size([100])


In [4]:
# hyperparameters
input_size = 784 # 28X28
hidden_size = 100
num_classes = 10
num_epochs = 1
learning_rate = 0.001
gamma = 1

In [5]:
ffn_model = FeedforwardNeuralNetModel(input_dim=input_size,
                                      hidden_dim=hidden_size,
                                      output_dim=num_classes,
                                      dropout_rate=0.5)

# define loss
criterion = FocalLoss(gamma = gamma)
# criterion = torch.nn.CrossEntropyLoss()

# define optimizer
optimizer = torch.optim.Adam(ffn_model.parameters(), lr=learning_rate)

# define scheduler for adjusting the learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min')
# scheduler = StepLR(optimizer, step_size = 4, gamma = 0.5)
# scheduler = None

In [6]:
# accuracy before training
pred, label = testing_pytorch(ffn_model, test_loader, criterion)
sum(pred==label) / len(pred)

Accuracy on dataset of size 10000: 10.770000457763672 %.
Average loss: 2.0794160413742064


tensor(0.1077)

In [7]:
ffn_model = training_pytorch(model=ffn_model,
                             train_loader=train_loader,
                             criterion=criterion,
                             optimizer=optimizer,
                             num_epochs=10,
                             scheduler=scheduler,
                             seed=seed,
                             verbose=True,
                             verbose_epoch=1)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1/10 || Item: 0/600 || Loss: 2.076205253601074
--------------------------------------------------
##### Epoch: 1/10 || Loss: 0.4600061774253845
--------------------------------------------------
Epoch: 2/10 || Item: 0/600 || Loss: 0.32463809847831726
--------------------------------------------------
##### Epoch: 2/10 || Loss: 0.14252567291259766
--------------------------------------------------
Epoch: 3/10 || Item: 0/600 || Loss: 0.18404611945152283
--------------------------------------------------
##### Epoch: 3/10 || Loss: 0.24533876776695251
--------------------------------------------------
Epoch: 4/10 || Item: 0/600 || Loss: 0.13073308765888214
--------------------------------------------------
##### Epoch: 4/10 || Loss: 0.19907833635807037
--------------------------------------------------
Epoch: 5/10 || Item: 0/600 || Loss: 0.20021291077136993
--------------------------------------------------
##### Epoch: 5/10 || Loss: 0.20535892248153687
-----------------------------

In [8]:
# accuracy after training
pred, label = testing_pytorch(ffn_model, test_loader, criterion)
sum(pred==label) / len(pred)

Accuracy on dataset of size 10000: 96.66999816894531 %.
Average loss: 0.07194365375558846


tensor(0.9667)